# Merging Datasets

This notebook details the process of merging multiple datasets into a single, unified dataset.  

In [1]:
import pandas as pd
import numpy as np
import dtale
from master_thesis.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR, load_dataframe_from_pickle, save_dataframe_as_pickle

2025-03-15 17:11:09.990 | INFO     | master_thesis.config:<module>:12 - PROJ_ROOT path is: /home/takosaga/Projects/master_thesis


In [2]:
prep_hatexplain_df = load_dataframe_from_pickle(INTERIM_DATA_DIR.as_posix() + '/prep_hatexplain_df.pkl')
prep_measuring_hate_speech_df = load_dataframe_from_pickle(INTERIM_DATA_DIR.as_posix() + '/prep_measuring_hate_speech_df.pkl')
prep_mlma_df = load_dataframe_from_pickle(INTERIM_DATA_DIR.as_posix() + '/prep_mlma_df.pkl')

In [3]:
prep_hatexplain_df.dtypes

text                                              object
original_label                                    object
original_target                                   object
original_id                                       object
platform                                          object
original_dataset_title                            object
label_hatespeech_binary_offensive_not_included    object
label_hatespeech_binary_offensive_included        object
label_normal_offensive_hatespeech                 object
dtype: object

In [4]:
prep_measuring_hate_speech_df.dtypes

original_id                                         int32
text                                               object
original_label                                    float64
platform                                           object
original_target                                    object
original_dataset_title                             object
label_hatespeech_binary_offensive_not_included     object
label_hatespeech_binary_offensive_included         object
label_normal_offensive_hatespeech                  object
dtype: object

In [5]:
prep_measuring_hate_speech_df['original_id'] = prep_measuring_hate_speech_df['original_id'].astype('object')
prep_measuring_hate_speech_df['original_label'] = prep_measuring_hate_speech_df['original_label'].astype('object')

In [6]:
prep_measuring_hate_speech_df.dtypes

original_id                                       object
text                                              object
original_label                                    object
platform                                          object
original_target                                   object
original_dataset_title                            object
label_hatespeech_binary_offensive_not_included    object
label_hatespeech_binary_offensive_included        object
label_normal_offensive_hatespeech                 object
dtype: object

In [7]:
prep_mlma_df.dtypes

original_id                                        int64
text                                              object
original_label                                    object
original_dataset_title                            object
platform                                          object
label_hatespeech_binary_offensive_not_included    object
label_hatespeech_binary_offensive_included        object
label_normal_offensive_hatespeech                 object
original_target                                   object
dtype: object

In [8]:
prep_mlma_df['original_id'] = prep_mlma_df['original_id'].astype('object')

In [9]:
prep_mlma_df.dtypes

original_id                                       object
text                                              object
original_label                                    object
original_dataset_title                            object
platform                                          object
label_hatespeech_binary_offensive_not_included    object
label_hatespeech_binary_offensive_included        object
label_normal_offensive_hatespeech                 object
original_target                                   object
dtype: object

In [10]:
print('Hatexplain shape:',prep_hatexplain_df.shape)
print('MLMA shape: ', prep_mlma_df.shape)
print('Measuring Hate Speech shape: ', prep_measuring_hate_speech_df.shape)

Hatexplain shape: (20148, 9)
MLMA shape:  (5646, 9)
Measuring Hate Speech shape:  (39495, 9)


In [11]:
merged_dfs = pd.concat([prep_hatexplain_df, prep_measuring_hate_speech_df, prep_mlma_df], ignore_index=True)

In [12]:
merged_dfs.head()

,text,original_label,original_target,original_id,platform,original_dataset_title,label_hatespeech_binary_offensive_not_included,label_hatespeech_binary_offensive_included,label_normal_offensive_hatespeech
0,i dont think im getting my baby them white 9 h...,normal,[None],1179055004553900032,twitter,HateXplain,not_hatespeech,normal,normal
1,we cannot continue calling ourselves feminists...,normal,[None],1179063826874032128,twitter,HateXplain,not_hatespeech,normal,normal
2,nawt yall niggers ignoring me,normal,[African],1178793830532956161,twitter,HateXplain,not_hatespeech,normal,normal
3,<user> i am bit confused coz chinese ppl can n...,hatespeech,[Asian],1179088797964763136,twitter,HateXplain,hatespeech,hatespeech/offensive,hatespeech
4,this bitch in whataburger eating a burger with...,hatespeech,"[Caucasian, Women]",1179085312976445440,twitter,HateXplain,hatespeech,hatespeech/offensive,hatespeech


In [13]:
d = dtale.show(merged_dfs)
d.open_browser()

In [14]:
d.kill()

2025-03-15 17:11:10,564 - INFO     - Shutdown complete


In [15]:
merged_dfs['platform'].value_counts()

platform
twitter    30148
reddit     15842
gab        11093
youtube     8178
nf_6           9
nf_2           8
nf_4           7
nf_3           2
nf_1           2
Name: count, dtype: int64

In [16]:
values_to_remove = ['nf_6', 'nf_2', 'nf_4', 'nf_3', 'nf_1']


In [17]:
merged_dfs = merged_dfs[~merged_dfs['platform'].isin(values_to_remove)]
merged_dfs['platform'].value_counts()

platform
twitter    30148
reddit     15842
gab        11093
youtube     8178
Name: count, dtype: int64

In [18]:
merged_dfs['original_dataset_title'].value_counts()

original_dataset_title
Measuring Hate Speech    39495
HateXplain               20120
MLMA                      5646
Name: count, dtype: int64

In [19]:
merged_dfs['original_target'].value_counts()

original_target
[None]                                                                                                                                               6569
[women, gender]                                                                                                                                      6335
[black, race]                                                                                                                                        2580
[African]                                                                                                                                            2271
[specific_country, origin]                                                                                                                           2118
                                                                                                                                                     ... 
[latinx, other, race, immigrant, other, origin]             

I want to extract original targets to simiplify to calls out real or perceived “identity factors” of an individual or a group, including: “religion, ethnicity, nationality, race, colour, descent, gender, but also characteristics such as language, economic or social origin, disability, health status, or sexual orientation, among many others.

In [20]:
# to get all the unique values in orginal_target
flattened_list = [item for sublist in merged_dfs['original_target'] for item in sublist if item]
flattened_list = [item.lower() for item in flattened_list]
all_targets = list(set(flattened_list))

In [21]:
all_targets

['atheist',
 'seniors',
 'homosexual',
 'indigenous',
 'economic',
 'immigrants',
 'african_descent',
 'refugee',
 'disability',
 'middle_aged',
 'unspecific',
 'indian/hindu',
 'left_wing_people',
 'specific_country',
 'non_binary',
 'teenagers',
 'buddhism',
 'immigrant',
 'black',
 'transgender_women',
 'hearing_impaired',
 'other',
 'special_needs',
 'age',
 'hispanics',
 'african',
 'arab',
 'pacific_islander',
 'christian',
 'arabs',
 'caucasian',
 'straight',
 'middle_eastern',
 'children',
 'physical',
 'mormon',
 'bisexual',
 'hindu',
 'young_adults',
 'sexual_orientation',
 'asian',
 'jewish',
 'men',
 'hispanic',
 'undocumented',
 'native_american',
 'lesbian',
 'indian',
 'transgender_unspecified',
 'buddhist',
 'religion',
 'cognitive',
 'jews',
 'white',
 'origin',
 'women',
 'transgender_men',
 'muslims',
 'race',
 'asians',
 'latinx',
 'gender',
 'individual',
 'islam',
 'none',
 'sexuality',
 'migrant_worker',
 'visually_impaired',
 'gay',
 'neurological',
 'refugees',

In [22]:
"""
For all the targets from 
'target_race_asian',
'target_race_black',
'target_race_latinx',
'target_race_middle_eastern',
'target_race_native_american',
'target_race_pacific_islander',
'target_race_white',
'target_race_other',
'target_race',
'target_religion_atheist',
'target_religion_buddhist',
'target_religion_christian',
'target_religion_hindu',
'target_religion_jewish',
'target_religion_mormon',
'target_religion_muslim',
'target_religion_other',
'target_religion',
'target_origin_immigrant',
'target_origin_migrant_worker',
'target_origin_specific_country',
'target_origin_undocumented',
'target_origin_other',
'target_origin',
'target_gender_men',
'target_gender_non_binary',
'target_gender_transgender_men',
'target_gender_transgender_unspecified',
'target_gender_transgender_women',
'target_gender_women',
'target_gender_other',
'target_gender',
'target_sexuality_bisexual',
'target_sexuality_gay',
'target_sexuality_lesbian',
'target_sexuality_straight',
'target_sexuality_other',
'target_sexuality',
'target_age_children',
'target_age_teenagers',
'target_age_young_adults',
'target_age_middle_aged',
'target_age_seniors',
'target_age_other',
'target_age',
'target_disability_physical',
'target_disability_cognitive',
'target_disability_neurological',
'target_disability_visually_impaired',
'target_disability_hearing_impaired',
'target_disability_unspecific',
'target_disability_other',
'target_disability'
"""

"\nFor all the targets from \n'target_race_asian',\n'target_race_black',\n'target_race_latinx',\n'target_race_middle_eastern',\n'target_race_native_american',\n'target_race_pacific_islander',\n'target_race_white',\n'target_race_other',\n'target_race',\n'target_religion_atheist',\n'target_religion_buddhist',\n'target_religion_christian',\n'target_religion_hindu',\n'target_religion_jewish',\n'target_religion_mormon',\n'target_religion_muslim',\n'target_religion_other',\n'target_religion',\n'target_origin_immigrant',\n'target_origin_migrant_worker',\n'target_origin_specific_country',\n'target_origin_undocumented',\n'target_origin_other',\n'target_origin',\n'target_gender_men',\n'target_gender_non_binary',\n'target_gender_transgender_men',\n'target_gender_transgender_unspecified',\n'target_gender_transgender_women',\n'target_gender_women',\n'target_gender_other',\n'target_gender',\n'target_sexuality_bisexual',\n'target_sexuality_gay',\n'target_sexuality_lesbian',\n'target_sexuality_straigh

The following made from the limitations of hurtlex lexicon

In [23]:
race_labels = ['asian', 'black', 'latinx', 'middle_eastern', 
                'native_american', 'pacific_islander', 'white', 'race']
gender_sexuality_labels = ['gender','men','non_binary', 'transgender_men', 'transgender_unspecified',
                'transgender_women', 'women', 'sexuality', 'bisexual', 'gay', 'lesbian', 'straight']
disability_labels = ['disability','physical','cognitive','neurological','visually_impaired',
                'hearing_impaired','unspecific']
other_lables = ['religion', 'atheist', 'buddhist', 'christian', 'hindu', 'jewish', 'mormon',
                'muslim', 'immigrant', 'migrant_worker', 'specific_country', 'undocumented', 'origin',
                'age', 'children', 'teenagers', 'young_adults', 'middle_aged', 'seniors']

In [ ]:
race = []
for label in race_labels:
    all_targets.remove(label)
    race.append(label)
    

In [25]:
gender_sexuality = []
for label in gender_sexuality_labels:
    all_targets.remove(label)
    gender_sexuality.append(label)
    


In [26]:
disability = []
for label in disability_labels:
    all_targets.remove(label)
    disability.append(label)


In [27]:

other = []
for label in other_lables:
    all_targets.remove(label)
    other.append(label)

In [28]:
all_targets

['homosexual',
 'indigenous',
 'economic',
 'immigrants',
 'african_descent',
 'refugee',
 'indian/hindu',
 'left_wing_people',
 'buddhism',
 'other',
 'special_needs',
 'hispanics',
 'african',
 'arab',
 'arabs',
 'caucasian',
 'sexual_orientation',
 'hispanic',
 'indian',
 'jews',
 'muslims',
 'asians',
 'individual',
 'islam',
 'none',
 'refugees']

In [38]:
more_race_labels = ['african_descent', 'indian/hindu', 'hispanics', 'african', 'arab', 'arabs', 'caucasian', 'hispanic', 'indian',
                    'asians']          

more_gender_sexuality_labels = ['homosexual', 'sexual_orientation']
more_disability_labels = ['special_needs']
more_other_labels = ['indigenous','economic', 'immigrants', 'refugee', 'left_wing_people','buddhism', 'other', 'jews', 'muslims', 
                    'individual', 'islam', 'refugees']

In [40]:
for label in more_race_labels:
    if label in all_targets:
        print(f"removing {label} from all_targets")
        all_targets.remove(label)
        race.append(label)
    else:
        print(f"{label} not found in all_targets")
        

removing african_descent from all_targets
removing indian/hindu from all_targets
removing hispanics from all_targets
removing african from all_targets
removing arab from all_targets
removing arabs from all_targets
removing caucasian from all_targets
removing hispanic from all_targets
removing indian from all_targets
removing asians from all_targets


In [41]:

for label in more_gender_sexuality_labels:
    if label in all_targets:
        print(f"removing {label} from all_targets")
        all_targets.remove(label)
        gender_sexuality.append(label)
    else:
        print(f"{label} not found in all_targets")


removing homosexual from all_targets
removing sexual_orientation from all_targets


In [42]:

for label in more_disability_labels:
    if label in all_targets:
        print(f"removing {label} from all_targets")
        all_targets.remove(label)
        disability.append(label)
    else:
        print(f"{label} not found in all_targets")


removing special_needs from all_targets


In [43]:

for label in more_other_labels:
    if label in all_targets:
        print(f"removing {label} from all_targets")
        all_targets.remove(label)
        other.append(label)
    else:
        print(f"{label} not found in all_targets")
        

removing indigenous from all_targets
removing economic from all_targets
removing immigrants from all_targets
removing refugee from all_targets
removing left_wing_people from all_targets
removing buddhism from all_targets
removing other from all_targets
removing jews from all_targets
removing muslims from all_targets
removing individual from all_targets
removing islam from all_targets
removing refugees from all_targets


In [44]:
all_targets

['none']

In [45]:
print(f"races: {race}")
print(f"gender_sexuality: {gender_sexuality}")
print(f"disability: {disability}")
print(f"other: {other}")

races: ['asian', 'black', 'latinx', 'middle_eastern', 'native_american', 'pacific_islander', 'white', 'race', 'african_descent', 'indian/hindu', 'hispanics', 'african', 'arab', 'arabs', 'caucasian', 'hispanic', 'indian', 'asians']
gender_sexuality: ['gender', 'men', 'non_binary', 'transgender_men', 'transgender_unspecified', 'transgender_women', 'women', 'sexuality', 'bisexual', 'gay', 'lesbian', 'straight', 'homosexual', 'sexual_orientation']
disability: ['disability', 'physical', 'cognitive', 'neurological', 'visually_impaired', 'hearing_impaired', 'unspecific', 'special_needs']
other: ['religion', 'atheist', 'buddhist', 'christian', 'hindu', 'jewish', 'mormon', 'muslim', 'immigrant', 'migrant_worker', 'specific_country', 'undocumented', 'origin', 'age', 'children', 'teenagers', 'young_adults', 'middle_aged', 'seniors', 'indigenous', 'economic', 'immigrants', 'refugee', 'left_wing_people', 'buddhism', 'other', 'jews', 'muslims', 'individual', 'islam', 'refugees']


In [48]:
def extract_target(target_list):
    if not target_list or target_list == ['None']:
        return {'none'}
    
    result = set()
    for item in target_list:
        item_lower = item.lower()
        if item_lower in race:
            result.add('race')
        elif item_lower in gender_sexuality:
            result.add('gender_sexuality')
        elif item_lower in disability:
            result.add('disability')
        elif item_lower in other:
            result.add('other')
    
    return result if result else {'none'}

In [49]:
merged_dfs['extracted_target'] = merged_dfs['original_target'].apply(extract_target)


In [50]:
merged_dfs.head()

,text,original_label,original_target,original_id,platform,original_dataset_title,label_hatespeech_binary_offensive_not_included,label_hatespeech_binary_offensive_included,label_normal_offensive_hatespeech,extracted_target
0,i dont think im getting my baby them white 9 h...,normal,[None],1179055004553900032,twitter,HateXplain,not_hatespeech,normal,normal,{none}
1,we cannot continue calling ourselves feminists...,normal,[None],1179063826874032128,twitter,HateXplain,not_hatespeech,normal,normal,{none}
2,nawt yall niggers ignoring me,normal,[African],1178793830532956161,twitter,HateXplain,not_hatespeech,normal,normal,{race}
3,<user> i am bit confused coz chinese ppl can n...,hatespeech,[Asian],1179088797964763136,twitter,HateXplain,hatespeech,hatespeech/offensive,hatespeech,{race}
4,this bitch in whataburger eating a burger with...,hatespeech,"[Caucasian, Women]",1179085312976445440,twitter,HateXplain,hatespeech,hatespeech/offensive,hatespeech,"{gender_sexuality, race}"


In [51]:
merged_dfs['extracted_target'].value_counts()


extracted_target
{gender_sexuality}                             18319
{other}                                        15594
{race}                                          9555
{none}                                          7715
{other, race}                                   5171
{other, gender_sexuality}                       3017
{gender_sexuality, race}                        2172
{disability}                                    1486
{other, gender_sexuality, race}                  831
{other, disability}                              693
{disability, gender_sexuality}                   406
{other, disability, gender_sexuality, race}      108
{disability, race}                                61
{other, disability, gender_sexuality}             61
{other, disability, race}                         45
{disability, gender_sexuality, race}              27
Name: count, dtype: int64